# Blue Brain BioExplorer

![](../../bioexplorer_banner.png)

### Initialization

In [ ]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, \
                        Cell, Sugar, MolecularSystemAnimationParams, Vector2, Vector3, Quaternion, MovieMaker
import glob
import os

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
mm = MovieMaker(be)
status = be.reset_scene()

### Resources

In [ ]:
resource_folder = os.path.abspath('../../../tests/test_files')

pdb_folder = os.path.join(resource_folder, 'pdb')
membrane_folder = os.path.join(pdb_folder, 'membrane')
lipids_folder = os.path.join(membrane_folder, 'lipids')

glucose_path = os.path.join(pdb_folder, 'glucose.pdb')
immune_folder = os.path.join(pdb_folder, 'immune')
lactoferrin_path = os.path.join(immune_folder, '1b0l.pdb')
defensin_path = os.path.join(immune_folder, '1ijv.pdb')

surfactant_folder = os.path.join(pdb_folder, 'surfactant')
surfactant_head_source = os.path.join(surfactant_folder, '1pw9.pdb')
surfactant_branch_source = os.path.join(surfactant_folder, '1k6f.pdb')

glycan_folder = os.path.join(pdb_folder, 'glycans')
complex_folder = os.path.join(glycan_folder, 'complex')
high_mannose_folder = os.path.join(glycan_folder, 'high-mannose')
hybrid_folder = os.path.join(glycan_folder, 'hybrid')
o_glycan_folder = os.path.join(glycan_folder, 'o-glycan')

complex_paths = [
    os.path.join(complex_folder, '33.pdb'),
    os.path.join(complex_folder, '34.pdb'),
    os.path.join(complex_folder, '35.pdb'),
    os.path.join(complex_folder, '36.pdb')
]
high_mannose_paths = [
    os.path.join(high_mannose_folder, '1.pdb'),
    os.path.join(high_mannose_folder, '2.pdb'),
    os.path.join(high_mannose_folder, '3.pdb'),
    os.path.join(high_mannose_folder, '4.pdb')
]
hybrid_paths = [os.path.join(hybrid_folder, '24.pdb')]
o_glycan_paths = [os.path.join(o_glycan_folder, '12.pdb')]

### Configuration

In [ ]:
# Scene
scene_size = Vector3(500.0, 500.0, 500.0)

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False

# Glycans
add_glycans = True
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS

## Camera and rendering settings

In [ ]:
status = core.set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[4.883, 44.255, 431.911],
    target=[4.883, 44.255, 31.311]
)

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

### Cell, SARS-COV-2 Coronavirus and Surfactant-D

In [ ]:
def add_virus(
        position, rotation, random_seed, random_position_seed,
        random_rotation_seed, morphing_step):
    be.add_sars_cov_2(
        name='sars-cov-2', resource_folder=resource_folder,
        atom_radius_multiplier=protein_radius_multiplier,
        representation=protein_representation,
        add_glycans=add_glycans,
        position=position,
        rotation=rotation,
        animation_params=MolecularSystemAnimationParams(
            random_seed, random_position_seed, 0.2, random_rotation_seed + 1, 0.025, morphing_step)
    )

In [ ]:
def add_cell(nb_receptors, random_seed, random_position_seed, random_rotation_seed):
    name='Cell'

    '''ACE2 receptor definition'''
    ace2_receptor = Protein(
        name=name + '_' + be.NAME_RECEPTOR,
        source=os.path.join(pdb_folder, '6m18.pdb'),
        occurrences=nb_receptors,
        transmembrane_params=Vector2(-6.0, 2.0))

    '''Membrane definition'''
    lipids = glob.glob(os.path.join(lipids_folder, '*.pdb'))[:4]
    membrane = Membrane(
        load_non_polymer_chemicals=True, load_bonds=True,
        lipid_sources=lipids,
        animation_params=MolecularSystemAnimationParams(
            random_seed, random_position_seed, 0.2, random_rotation_seed, 0.025)
        )

    '''Cell definition'''
    cell_size = Vector3(scene_size.x, scene_size.y / 10.0, scene_size.z)
    cell = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_SINUSOID,
        shape_params=cell_size,
        membrane=membrane, 
        proteins=[ace2_receptor]
    )

    '''Add cell to scene'''
    status = be.add_cell(
        cell=cell, representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=Vector3(0.0, -80.0, 0.0)
    )

    if add_glycans:
        '''Glycans'''
        be.add_multiple_glycans(
            representation=protein_representation, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[53, 90, 103, 322, 432, 690],
            animation_params=MolecularSystemAnimationParams(
                random_seed, random_position_seed, 0.2, random_rotation_seed, 0.025))

        be.add_multiple_glycans(
            representation=protein_representation, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[546],
            animation_params=MolecularSystemAnimationParams(
                random_seed, random_position_seed, 0.2, random_rotation_seed, 0.025))

        indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugar(
                assembly_name=name, name=o_glycan_name, 
                source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, 
                representation=protein_representation,
                chain_ids=[2, 4], site_indices=[index[0]], 
                rotation=index[1],
                animation_params=MolecularSystemAnimationParams(
                    random_seed, random_position_seed, 0.2, random_rotation_seed, 0.025))
            be.add_sugar(o_glycan)
    return status

In [ ]:
def add_surfactant_d(name, position, rotation, animation_params):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    return be.add_surfactant(
        surfactant=surfactant_d, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, rotation=rotation, 
        animation_params=animation_params)

In [ ]:
def add_glucose(random_seed, nb_glucoses=4 * 14400):
    volume_position = Vector3(0.0, scene_size.y / 2.0 - 80.0, 0.0)
    glucose = Protein(
        name=be.NAME_GLUCOSE,
        source=glucose_path, load_non_polymer_chemicals=True, 
        animation_params=MolecularSystemAnimationParams(
            1, random_seed + 1, scene_size.y / 500.0, random_seed + 2, 1.0),
        occurrences=nb_glucoses)
    volume = Volume(
        name=be.NAME_GLUCOSE,
        shape=be.ASSEMBLY_SHAPE_CUBE,
        shape_params=scene_size, 
        protein=glucose
    )
    return be.add_volume(
        volume=volume, 
        representation=be.REPRESENTATION_ATOMS_AND_STICKS,
        atom_radius_multiplier=protein_radius_multiplier,
        position=volume_position)

In [ ]:
def set_materials():
    be.apply_default_color_scheme(
        shading_mode=be.SHADING_MODE_BASIC, user_parameter=1.0)
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']
        if be.NAME_COLLAGEN in model_name:
            material_ids = list(be.get_material_ids(model_id)['ids'])
            nb_materials = len(material_ids)
            palette = list()
            emissions = list()
            for i in range(nb_materials):
                palette.append([1,1,1])
                emissions.append(0.1)
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette,
                emissions=emissions
            )

## Movie

### Camera positions

In [ ]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 0.0, -1.0],
        'focusDistance': 0.0,
        'origin': [8.848612545524386, 38.1815273571507, 544.0734702858679],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.6083991714336657, -0.15450233390242366, -0.7784468363462806],
        'focusDistance': 0.0,
        'origin': [-240.8190994911038, 53.51713296185992, 342.1357352156152],
        'up': [0.07492786634686985, 0.9876682399356482, -0.13746731490476025]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.8612481222991739, -0.22198006941973533, 0.4571394979834462],
        'focusDistance': 0.0,
        'origin': [-337.42978759761564, 82.7875998078893, -205.1502287344292],
        'up': [0.21224592777192716, 0.9744622627257351, 0.0733141505282188]
    }
]
mm.build_camera_path(key_frames, 300, 50)

### Frames

In [ ]:
'''sars-cov-2'''
q_c_start = Quaternion(0.707, 0.707, 0.0, 0.0)
q_c_end = Quaternion(1.0, 0.0, 0.0, 0.0)

'''Surfactant-D'''
q_s_start = Quaternion(0.0, 0.0, 0.707,0.707)
q_s_end = Quaternion(1.0, 0.0, 0.0, 0.0)
q_s_nb_frames = 600

In [ ]:
from tqdm import tqdm

'''Settings'''
output_folder = '/tmp'

image_size = [960, 540]
image_samples_per_pixel = 64
nb_frames = mm.get_nb_frames()

'''Virus flying information'''
flying_nb_frames = 200

'''Virus landing information'''
landing_nb_frames = 50
landing_distance = 40.0

'''Virus merging information'''
merging_nb_frames = 100

be.set_general_settings(model_visibility_on_creation=False, mesh_folder='/tmp')
core.set_application_parameters(image_stream_fps=0)

for frame in tqdm(range(nb_frames)):
    '''Camera'''
    mm.set_current_frame(frame)
    
    '''Cell'''
    add_cell(5, 2, 5 * frame + 2, 5 * frame + 3)
    
    '''sars-cov-2'''
    show_virus = True
    morphing_step = 0.0
    p = Vector3(55.0, -5.0, -45.0)
    o = q_c_end
    if frame <= flying_nb_frames:
        '''Flying'''
        p = Vector3(55.0, -5.0 + flying_nb_frames - frame, -45.0 - (frame - flying_nb_frames) / 2.0)
        o = Quaternion.slerp(q_c_start, q_c_end, (frame + 1) / flying_nb_frames)
    elif frame <= landing_nb_frames + flying_nb_frames:
        '''Landing...'''
        progress = float(frame - flying_nb_frames) * (landing_distance / float(landing_nb_frames))
        p = Vector3(55.0, -5.0 - progress, -45.0)

        '''Modify receptor position to dive into the cell membrane'''
        be.set_protein_instance_transformation(
            assembly_name='Cell',
            name='Cell_' + be.NAME_RECEPTOR,
            instance_index=4,
            position=Vector3(74.5, -76.513 - progress, -39.5),
            rotation=Quaternion(-0.0385459, -0.0398906, 0.718016, 0.693813)
        )            

    elif frame <= landing_nb_frames + flying_nb_frames + merging_nb_frames:
        '''Merging...'''
        p = Vector3(55.0, -5.0 - landing_distance, -45.0)
        morphing_step = float(frame - flying_nb_frames - landing_nb_frames) / float(merging_nb_frames)
    else:
        be.remove_assembly('sars-cov-2')
        show_virus = False

    if show_virus:
        add_virus(
            position=p, rotation=Quaternion(o[0],o[1],o[2],o[3]),
            random_seed=3, 
            random_position_seed=5 * frame + 5,
            random_rotation_seed=5 * frame + 6,
            morphing_step=morphing_step)            

    '''Surfactant-D'''
    o = Quaternion.slerp(
        q_s_start, q_s_end, (frame + 1) / q_s_nb_frames)
    add_surfactant_d(
        name='Surfactant-D 1', 
        position=Vector3(
            -150.0 + (nb_frames - frame) / 7.0,
            22.0 + (nb_frames - frame) / 10.0, 
            -50.0 + (nb_frames - frame) / 5.0),
        rotation = Quaternion(o[0],o[1],o[2],o[3]),
        animation_params=MolecularSystemAnimationParams(seed=1)
    )

    '''Glucose'''
    add_glucose(frame)

    '''Materials'''
    set_materials()
    
    '''Make all models visible'''
    be.set_models_visibility(True)
    
    '''Snapshot'''
    mm.create_snapshot(
        renderer='bio_explorer',
        size=image_size, samples_per_pixel=image_samples_per_pixel,
        path=output_folder, base_name='%05d' % frame)

core.set_application_parameters(image_stream_fps=20)